In [105]:

import pandas as pd
import numpy as np

# для работы с адресами
from geopy.geocoders import Nominatim
#для работы с системными файлами
import os
#для работы с регулярными выражениями
import re
import time

### Сбор данных

___

**Так как данные поступали в течени года и можно было с ними начать работать с самого начала проекта, было создано несколько функций для сбора всей информации в один датасэт**

In [106]:
DIRECTORY_fold = 'data/'

def get_list_all_files() -> list:
    """
    Returns:
        list: функция возвращает названия всех xlsx документов в нужном формате из папки data
    """
    
    ALL_FILES = []
    folds = os.listdir(DIRECTORY_fold)
    for fold in folds:
        DIRECTORY_file = DIRECTORY_fold + str(fold)
        files = os.listdir(DIRECTORY_file)
        files = os.listdir(DIRECTORY_file)
        ALL_FILES.extend(files)
    
    return ALL_FILES


def conversion(df):
    if len(df.columns) > 10:
        raise TypeError('Не подходящий формат')
    else:
        cols = df.columns[:8]
        df = df[cols]
        df['Unnamed: 3'] = df['Unnamed: 3'].apply(lambda x: np.nan if x == 'Наименование' else x)
        df.drop(df.columns[:2], axis=1, inplace=True)
        df.dropna(axis=0, inplace=True)
        df.set_index(np.arange(len(df.index)), inplace=True)

        #переименовываем столбцы
        df.rename(columns={'Unnamed: 2':'num', 'Unnamed: 3':'ration', 'Unnamed: 4':'address',
                        'Unnamed: 5':'start', 'Unnamed: 6':'finish',
                        'Unnamed: 7':'comment'}, inplace=True)

        # функция которая сокращает название рациона
        def edit_ration_name(name):
            if 'Снижение' in name:
                return name.replace('Снижение', '').strip()
            elif 'Баланс' in name:
                return name.replace('Баланс', '').strip()
            elif 'Classic' in name:
                return name.replace('Classic', '').strip()
            elif 'Набор' in name:
                return name.replace('Набор', '').strip()
            

        df['start'] = df['start'].apply(lambda x: int(x[0]) if len(x) <= 4 else int(x[:2])) # заменяем начало интервала со строкого формата на числовой
        df['finish'] = df['finish'].apply(lambda x: int(x[0]) if len(x) <= 4 else int(x[:2]))# заменяем конец интервала со строкого формата на числовой
        df['ration'] = df['ration'].apply(edit_ration_name)
    
        return df


def get_df(files: list) -> pd.DataFrame:
    """
    Args:
        files (list): Список названий файлов

    Returns:
        pd.DataFrame: датасэт из всех xlsx файлов в формате DataFrame
    """
    for i, file in enumerate(files):
        try:
            month = int(file.split('.')[1])
        except ValueError:
            print(file)
        if i == 0:
            data = pd.read_excel(f'data/{month}/{file}')
            data = conversion(data)
            data['date'] = [file.split('.')[:2]] * len(data)
            lenght = len(data)
        else:
            try:
                data_plus = pd.read_excel(f'data/{month}/{file}')
                data_plus = conversion(data_plus)
                data_plus['date'] = [file.split('.')[:2]] * len(data_plus)
                lenght += len(data_plus)
                data = pd.concat([data, data_plus], axis=0)
            except Exception:
                continue
    
    #data = data.drop(['Unnamed: 9'], axis=1)
    print(f'data shape is {data.shape[0]} should equal {lenght}')

    return data

Проверка формата каждого файла

In [107]:
flag_2 = True

if flag_2 is True:
    res = {}
    folds = os.listdir(DIRECTORY_fold)
    for fold in folds:
        DIRECTORY_file = DIRECTORY_fold + str(fold)
        files = os.listdir(DIRECTORY_file)
    
        ls = []
        for file in files:
            some_name = pd.read_excel(f'data/{fold}/{file}')
            finall = [file.split('.')[0], some_name.shape[1]]
            ls.append(finall)
        res[int(fold)] = ls

    for m, val in res.items():
        for elem in val:
            if elem[1] > 8:
                print(f'Month {m}')
                print(elem[0])

Month 6
25
Month 2
13
Month 4
09
Month 4
30
Month 8
29
Month 7
31
Month 5
03


___

In [108]:
ALL_FILES = get_list_all_files()
df = get_df(ALL_FILES)

/tmp/ipykernel_15153/1721267670.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unnamed: 3'] = df['Unnamed: 3'].apply(lambda x: np.nan if x == 'Наименование' else x)
/tmp/ipykernel_15153/1721267670.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.columns[:2], axis=1, inplace=True)
/tmp/ipykernel_15153/1721267670.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

data shape is 162894 should equal 162894


**was 160180**

### Преоброзование датасэта

___

In [109]:
df_test = df.copy(deep=True)

In [110]:
# Признак date преобразуем в формат datetime
df_test['date'] = df_test['date'].apply(lambda x: '/'.join(x) + '/2022')
df_test['date'] = pd.to_datetime(df_test['date'], format='%d/%m/%Y')

# Выделяем отдельные признаки месяца, дня месяца и дня недели
df_test['month'] = df_test['date'].dt.month
df_test['day'] = df_test['date'].dt.day
df_test['dow'] = df_test['date'].dt.day_of_week

In [111]:
# В ноябре названия некоторых пакетов поменялись, но сущность осталось таже. Поэтому переименовываем их обратно в старые, чтобы лучше ориентироваться
ration_new_dict = {
    '5 блюд 3 дня': 'M',
    '6 блюд 3 дня': 'L',
    '4 блюда 3 дня': 'S',
    '3 блюда 3 дня': 'XS',
    '5 блюд 2 дня': 'M (2)',
    '4 блюда 2 дня': 'S (2)',
    '6 блюд 2 дня': 'L (2)',
    '3 блюда 2 дня': 'XS (2)'
    }

df_test['ration'] = df_test['ration'].apply(lambda x: ration_new_dict[x] if x in ration_new_dict.keys() else x)

In [112]:
df_test = df_test.drop(['num', 'comment'], axis=1)
df_test.reset_index(inplace=True, drop=True)

In [113]:
def get_location(df: pd.DataFrame) -> list:
    """ Функция возвращает которкий адрес
        для улучшения поиска в библиотеке geopy

    Args:
        df (pd.DataFrame): Датафрейм с полными адресами включая квартиру, этаж и т.д

    Returns:
        list: Возвращает два списка: 
                                    1. C улицей и номером дома
                                    2. Населенный пункт
    """
    #список адресов
    list_of_address = df['address'].to_list()

    #созадем списки улицы, номера дома и города
    street_list, area_ekb_final = [], []
    
    #достаем названия улиц
    for elem in list_of_address:
        dig = re.findall(r'\d+', elem)
        ls = elem.split(',')
        if dig != []:
            result = [' '.join(ls[ind-1: ind + 1]) for ind, part in enumerate(ls) if dig[0] in part]
        else:
            result = 'not_detected'
        street_list.append(result)
    
    for ind, elem in enumerate(street_list):
        if elem == '':
            result = elem.split(',')[1:3]
            street_list[ind] = ' '.join(result)
    
    street_list = [elem[1].strip() if elem[0] == ' ' else elem[0].strip() for elem in street_list]
    street_list = [elem.replace('улица', 'ул').replace('проспект', 'пр-кт') for elem in street_list]
    
    #----------------------------------------------------------------
    # Достаем из полного адреса населенный пункт

    # Список крупных населенных пунктов
    area_ekb = [
    'Екатеринбург','Верхняя Пышма', 'Арамиль', 'Березовский', 'Ревда',
    'Среднеуральск', 'Полевской', 'Берёзовский', 'Заречный', 'Сысерть'
    ]

    # Проходимся по каждому адресу и достаем населенный пункт.
    # Сначала идет проверка на крупные наеленные пункты,
    # затем остается проверка на небольшие деревни и села.
    for elem in list_of_address:
        elem_list = [val.strip() for val in elem.split(',')]
        elem_set = set(elem_list)
        result = elem_set.intersection(area_ekb)
        if result == set():
            result = re.findall(r'снт \w+,', elem.lower())
            if result == []:
                result = re.findall(r'поселок \w+,', elem.lower())
                if result == []:
                    result = re.findall(r'село \w+,', elem.lower())
                    if result == []:
                        result ='Екатеринбург'
                    
        area_ekb_final.append(''.join(result))


    return street_list, area_ekb_final

In [114]:
street_list, area_ekb_exp = get_location(df_test)

df_test['street'] = street_list
df_test['area'] = area_ekb_exp
df_test['short_address'] = df_test['area'] + ', ' + df_test['street']

In [115]:
df_test

,ration,address,start,finish,date,month,day,dow,street,area,short_address
0,1000,"Среднеуральская улица, 1, Ревда, Свердловская ...",6,8,2022-10-26,10,26,2,Среднеуральская ул 1,Ревда,"Ревда, Среднеуральская ул 1"
1,750,"подъезд 2, улица Чехова, 21, Ревда, Свердловск...",6,8,2022-10-26,10,26,2,,Ревда,"Ревда,"
2,1500,"Россия, Свердловская область, Ревда, Рябиновая...",6,8,2022-10-26,10,26,2,Рябиновая ул 3,Ревда,"Ревда, Рябиновая ул 3"
3,1500,"Россия, Свердловская область, Первоуральск, Бе...",6,8,2022-10-26,10,26,2,Береговая ул 80,Екатеринбург,"Екатеринбург, Береговая ул 80"
4,1500,"подъезд 3, улица Емлина, 16А, Первоуральск, Св...",6,8,2022-10-26,10,26,2,,Екатеринбург,"Екатеринбург,"
...,...,...,...,...,...,...,...,...,...,...,...
162889,2000,"Свердловская область, Екатеринбург, Кузнечная ...",8,10,2022-05-01,5,1,6,Кузнечная ул 83,Екатеринбург,"Екатеринбург, Кузнечная ул 83"
162890,750,"Екатеринбург Кузнечная улица 81 , кв. 58, эт. ...",10,12,2022-05-01,5,1,6,,Екатеринбург,"Екатеринбург,"
162891,S,"Россия, Свердловская область, Екатеринбург, Ур...",10,12,2022-05-01,5,1,6,Уральская ул 5,Екатеринбург,"Екатеринбург, Уральская ул 5"
162892,1500,"Россия, Свердловская область, Екатеринбург, Тр...",10,12,2022-05-01,5,1,6,Трамвайный переулок 2к7,Екатеринбург,"Екатеринбург, Трамвайный переулок 2к7"


**Каждому адресу присваеваем широту и долготу**

Чтобы присвоить каждому адресу координаты, я решил использовать библиотеку *geopy*. Но если напрямую использовать данную библиотеку через метод *apply*, то разметка геоданных займет около 18 часов. Это связанно из-за невозможности распараллеливать вычисления(разрешен 1 запрос в n-время) и из-за траты времени на обработку запроса в диапазоне от 1 до 2 секунд. 

Поэтому было принято решение создать словарь, где ключ был бы равен какому-то адресу, а значение - это геоданные данного адреса. Так обработанный запрос записывался бы в словарь и уже при новом запросе с похожим адресом обращался бы к нему.

In [116]:
# Данный словарь уже у нас записан, поэтому

#Создаем объект класса для Nominatim из библиотеки geopy
geolocator = Nominatim(user_agent='location')
#Загружаем наш словарь(он уже заполнен некторыми адресами)
dict_loc= np.load('secondary_data/dict_loc.npy', allow_pickle=True).item()
#Геоданные Екатеринбурга, если geopy не распознает наш адрес
ekb_loc = [56.838011, 60.597474]

def get_geolocation(st: str) -> list:
    """Функция возвращает координаты входного адреса

    Args:
        st (str): Адрес дома

    Returns:
        list: Список координат 
    """
    try:
        result = dict_loc[st]
    # Если нет данного адреса в словаре, то срабатывает исключение
    except KeyError:
        try:
            location = geolocator.geocode(st, timeout=None)
            dict_loc.setdefault(st, [location.latitude, location.longitude])
            result = dict_loc[st]
        # Если geocode не распознал адрес,
        # то срабатывает исключение и забиваются координаты Екатеринбурга
        except AttributeError:
            dict_loc.setdefault(st, ekb_loc)
            result = ekb_loc 
    
    return result

In [117]:
# Поставить True, если нужно запустить ячейку
# Данная ячейка дополняет наш словарь новыми адресами,
# Она была использована для первичного заполнения словаря, когда он был пуст
flag = True

if flag is True:
    #Показывает количесвто уникальных адресов с каждой итерацией
    dict_density = []
    # Условие при котором можно остановить цикл поиска уникальных ключей,
    # достаточное для разметки данных в адекватный промежуток времени
    CONDITION = df_test['short_address'].nunique() * 0.8

    # Для быстрого поиска уникальных адресов, решено разбить датасэт на выборки из 10 частей
    split_data = np.array_split(df_test, 10)

    
    while len(dict_loc) <= CONDITION:
        #Проход по всем выборкам происходит со смещением в середину.
        # Это сделано для того, чтобы как можно быстрее охватить новые адреса и включить их в словарь,
        # так как за год новые адреса(клиенты) появлялись постоянно, а старые могли уходить.
        for i in [0, 9, 1, 8, 2, 7, 3, 6, 4, 5]:
            # Чтобы не прочитывать всю выборку, решено взять лишь часть в размере 33% 
            var_data = split_data[i].sample(3000)
            var_data['loc'] = var_data['short_address'].apply(get_geolocation)
            dict_density.append(len(dict_loc))
        
        #Сохраняем наш словарь
        np.save('secondary_data/dict_loc.npy', dict_loc)
        print(f'Уникальных объектов: {dict_density[-1]}, За цикл прибавилось: {max(dict_density) - min(dict_density)}')

In [118]:
start = time.time()
df_test['location'] = df_test['short_address'].apply(get_geolocation)
end = time.time()
print(f'Обработка адресов заняло {int((end - start) / 60)} минут(-ы)')

Обработка адресов заняло 3 минут(-ы)


**В итоге вместо 18 часов, добавление координат занимает несколько минут**

In [119]:
location_list = df_test['location'].to_list()
df_test['latitude'] = [elem[0] for elem in location_list]
df_test['longitude'] = [elem[1] for elem in location_list]

**Можно привязать погоду к каждой дате, чтобы увидеть в будущем какие либо закономерности**

In [120]:
weather = pd.read_csv('secondary_data/weather.csv', sep=';')
weather.head(3)

,Местное время в Екатеринбурге,T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
31.12.2022 23:00,-24.2,747.2,776.5,-0.3,73,"Ветер, дующий с востока",1,NaN,NaN,70 – 80%.,...,"Перистых, перисто-кучевых или перисто-слоистых...",10.0,-27.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31.12.2022 20:00,-23.2,747.5,776.6,0.0,73,"Штиль, безветрие",0,NaN,NaN,70 – 80%.,...,"Перистые нитевидные, иногда когтевидные, не ра...",10.0,-26.7,Следы осадков,12.0,NaN,NaN,NaN,NaN,NaN
31.12.2022 17:00,-22.2,747.5,776.5,0.1,75,"Штиль, безветрие",0,NaN,NaN,60%.,...,Перистые (часто в виде полос) и перисто-слоист...,10.0,-25.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [121]:
# Нам нужна только температура из данной таблицы
date, forecast = weather['Местное время в Екатеринбурге'].index, weather['Местное время в Екатеринбурге'].values
weather = pd.DataFrame({'date': date, 'weather': forecast})
weather['date'] = pd.to_datetime(weather['date']).dt.date

#Групируем данные по дате и находим среднюю температуру
weather = weather.groupby('date', as_index=False).mean()
weather['date'] = pd.to_datetime(weather['date'])

In [122]:
# Добовляем нашу температуру к главной таблице
df_test = df_test.merge(weather, how='left')
df_test['weather'] = df_test['weather'].apply(int)
df_test.head()

,ration,address,start,finish,date,month,day,dow,street,area,short_address,location,latitude,longitude,weather
0,1000,"Среднеуральская улица, 1, Ревда, Свердловская ...",6,8,2022-10-26,10,26,2,Среднеуральская ул 1,Ревда,"Ревда, Среднеуральская ул 1","[56.8519689, 59.9111315]",56.851969,59.911132,0
1,750,"подъезд 2, улица Чехова, 21, Ревда, Свердловск...",6,8,2022-10-26,10,26,2,,Ревда,"Ревда,","[56.8000651, 59.9086688]",56.800065,59.908669,0
2,1500,"Россия, Свердловская область, Ревда, Рябиновая...",6,8,2022-10-26,10,26,2,Рябиновая ул 3,Ревда,"Ревда, Рябиновая ул 3","[56.774023, 59.91621199358074]",56.774023,59.916212,0
3,1500,"Россия, Свердловская область, Первоуральск, Бе...",6,8,2022-10-26,10,26,2,Береговая ул 80,Екатеринбург,"Екатеринбург, Береговая ул 80","[56.8789439, 60.4048658]",56.878944,60.404866,0
4,1500,"подъезд 3, улица Емлина, 16А, Первоуральск, Св...",6,8,2022-10-26,10,26,2,,Екатеринбург,"Екатеринбург,","[56.839104, 60.60825]",56.839104,60.608250,0


### Работа с пропусками

In [123]:
df_test.isnull().sum()

ration           1580
address             0
start               0
finish              0
date                0
month               0
day                 0
dow                 0
street              0
area                0
short_address       0
location            0
latitude            0
longitude           0
weather             0
dtype: int64

**В признаке *ration* отсутствуют данные для 1000 строк, заполних им.** 

**Также создадим несколько дополнительных признаков - это цена за каждый пакет и строковое значение месяца**

In [124]:
price_list ={
    '1500': 650, '1000': 600, '2000': 700, '750': 550, '2500': 770,
    'M': 610, 'L': 670, 'S': 550, '3500': 850, 'XS': 500, 'M (2)': 320, 'L (2)': 330,
    '1500 (1)': 330, '1000 (1)': 300, 'S (2)': 360, '2000 (1)': 350,
    '750 (1)': 280, 'XS (2)': 330, '2500 (1)': 390, '1200': 630,
    '3500 (1)': 430, '1800': 680, '2400': 750
    }

month_dict = {
    1:'Январь', 2:'Февраль', 3:'Март', 4:'Апрель',
    5:'Май', 6:'Июнь', 7:'Июль', 8:'Август',
    9:'Сентябрь', 10:'Октябрь', 11:'Ноябрь', 12:'Декабрь'
    }

In [125]:
# Заполняем пропуски
ration_list = list(price_list.keys())
df_test['ration'] = df_test['ration'].apply(lambda x: np.random.choice(ration_list) if x == None else x)

# Пропусков больше нет
df_test.isnull().sum()

ration           0
address          0
start            0
finish           0
date             0
month            0
day              0
dow              0
street           0
area             0
short_address    0
location         0
latitude         0
longitude        0
weather          0
dtype: int64

In [131]:
df_test['ration'] = df_test['ration'].apply(lambda x:x.replace('зеленая', '').strip() if 'зеленая' in x else x)
df_test['ration'] = df_test['ration'].apply(lambda x:'1500 (1)' if 'Меню' in x else x)

In [132]:
# Создаем признаки цены и строкого значения месяца
df_test['price'] = df_test['ration'].map(price_list)
df_test['month_str'] = df_test['month'].map(month_dict)

In [133]:
# Основная таблица отсортированна и готова к анализу
df = df_test.sort_values(by='date').reset_index(drop=True)
df.head()

,ration,address,start,finish,date,month,day,dow,street,area,short_address,location,latitude,longitude,weather,price,month_str
0,1500,"Россия, Свердловская область, Екатеринбург, ул...",9,11,2022-01-02,1,2,6,ул Фурманова 125,Екатеринбург,"Екатеринбург, ул Фурманова 125","[56.8118161, 60.58459613144561]",56.811816,60.584596,-5,650,Январь
1,3500,"Екатеринбург, ул Черничная, 46",9,11,2022-01-02,1,2,6,ул Черничная 46,Екатеринбург,"Екатеринбург, ул Черничная 46","[56.804057, 60.49024459500433]",56.804057,60.490245,-5,850,Январь
2,M,"Екатеринбург, поселок Мичуринский, ул Широкоре...",9,11,2022-01-02,1,2,6,ул Широкореченская 49,Екатеринбург,"Екатеринбург, ул Широкореченская 49","[56.7932523, 60.47724500990231]",56.793252,60.477245,-5,610,Январь
3,L,"Екатеринбург, поселок Мичуринский, ул Широкоре...",9,11,2022-01-02,1,2,6,ул Широкореченская 49,Екатеринбург,"Екатеринбург, ул Широкореченская 49","[56.7932523, 60.47724500990231]",56.793252,60.477245,-5,670,Январь
4,3500,"Свердловская область, Екатеринбург, микрорайон...",10,12,2022-01-02,1,2,6,ул Академика Бардина 5к1,Екатеринбург,"Екатеринбург, ул Академика Бардина 5к1","[56.8055187, 60.5595781]",56.805519,60.559578,-5,850,Январь


In [149]:
df['period'] = df.groupby('address', as_index=False)['date'].diff()
df['period'] = df['period'].dt.days
df['period'].fillna(-1, inplace=True)
df['period'] = df['period'] + 1

___

In [150]:
ls_clietns = df['address'].unique()

In [151]:
id_ls = []
id_dict = {}
def gen_id():
    return np.random.choice(['j', 'k', 's', 'x']) + str(np.random.randint(10000, 99999))

for client in ls_clietns:
    
    id_ = gen_id()
    while id_ in id_ls:
        id_ = gen_id()
    
    id_ls.append(id_)
    id_dict[client] = id_dict.setdefault(client, id_)

df['id'] = df['address'].map(id_dict) 

In [152]:
clients = df.groupby('id', as_index=False)['address'].count()

In [153]:
clients.drop('address', axis=1, inplace=True)
clients = clients.merge(df[['id','short_address', 'latitude','longitude']])
clients.drop_duplicates(inplace=True)

In [154]:
df['time_zone'] = df['start'].map({6:1, 7:2, 8:3, 9:4, 10:5, 11:6})

In [155]:
time_zone = df.groupby('id', as_index=False)['time_zone'].median()
clients = clients.merge(time_zone)
clients['time_zone'] = clients['time_zone'].map({1:'6-8', 2:'7-9', 3:'8-10', 4:'9-10', 5:'10-11', 6:'11-12'})

In [156]:
sum_trans = df.groupby('id', as_index=False)['weather'].count()
clients = clients.merge(sum_trans)
clients = clients.rename(columns={'weather':'sum_trans'})

In [157]:
period = df.groupby('id', as_index=False)['period'].median()
clients = clients.merge(period)

In [158]:
clients.head(5)

,id,short_address,latitude,longitude,time_zone,sum_trans,period
0,j10024,"Екатеринбург, ул Евгения Савкова 4",56.796193,60.493775,9-10,3,5.0
1,j10110,"Екатеринбург, ул Данилы Зверева 17а",56.857486,60.653772,6-8,1,0.0
2,j10142,"Екатеринбург, ул Евгения Савкова 4",56.796193,60.493775,6-8,3,4.0
3,j10158,"Екатеринбург, ул Горняков 18",56.872069,60.295732,8-10,1,0.0
4,j10178,"село косулино,, Приречная ул 13Б",56.838011,60.597474,6-8,10,4.0


In [164]:
df.to_csv('suport/transactions.csv', index=False)
clients.to_csv('suport/clients.csv', index=False)